# Protfolio optimization with CIM

## Equations

### Optimization problem formulation

Potential:
$$
V(\omega; \mu, \Theta) = \sum_{ij}^N -\mu_i \omega_i + \frac{\gamma}{2} \Theta_{ij} \omega_i \omega_j 
$$
Constraints:


-Allocations (Box constraints)
$$
\sum_{i \in I_k} \omega_i \leq \lambda_{I_k}
$$
for some subsets $I_k \subseteq \left[ N \right]$, whre $k \in [M']$ 

-Budget constraints 
$$
\sum_{i \in J_k} \omega_i = \lambda_{J_k}
$$
for some subsets $J_k \subseteq \left[ N \right]$, whre $k \in [M]$ , with $\lambda_{\left[ N \right]} = \lambda_0 = 1$

-Continuous formulation of budget constraints (with lagrange multiplier $\rho$)
$$
U(\omega; u, \lambda) = -\rho \sum_{k=0}^M \sum_i \left[ (u_k)_i \omega_i - \lambda_k \right]^2
$$
where $(u_k)_i = 1$ if the $i^{\text{th}}$ asset is in $J_k$, $0$ otherwise.

-Trading costs (backwards, real-life)
$$
\underset{\omega_i(t)}{\text{min}} \underbrace{\sum_i^N \left| \omega_i (t) - \overbrace{\omega_i(t - \delta t)}^{\equiv \Omega_i} \right| \nu_i }_{ \equiv W(\omega; \nu, \Omega) }
$$
-Trading costs (modified to have continuous derivative)
$$
\underset{\omega_i(t)}{\text{min}} \underbrace{\sum_i^N \left( \omega_i  - \Omega_i \right)^2\nu_i }_{ \equiv W'(\omega; \nu, \Omega) }
$$
-Trading cost, each transaction has a fix additional cost

### Gradient rule
$$
\frac{d \omega_k}{dt} = -\frac{\partial_k }{\partial \omega_k } \left[ V(\omega; \mu, \Theta) + W'(\omega; \nu, \Omega) + U(\omega; u, \lambda)\right]
$$

Projection to enforce box constraints...


$$
\begin{aligned}

- \frac{\partial }{\partial \omega_k }  V(\omega; \mu, \Theta) &= - \partial_k \left( \sum_{ij}^N -\mu_i \omega_i + \frac{\gamma}{2} \Theta_{ij} \omega_i \omega_j  \right) \\

 &= - \sum_{ij}^N -\mu_i \delta_{i,k} + \frac{\gamma}{2} \Theta_{ij} \delta_{i,k} \omega_j + \frac{\gamma}{2} \Theta_{ij} \omega_i \delta_{k, j}  \\

 &= \sum_{i}^N \mu_k  - \gamma \Theta_{ik} \omega_i \\

 &= \mu_k - \gamma \sum_{i}^N \Theta_{ik} \omega_i \\
\end{aligned}
$$

$$

\begin{aligned}
-\frac{\partial }{\partial \omega_k } W'(\omega; \nu, \Omega) &= - \partial_k \sigma \sum_i (\omega_i - \Omega_i)^2 \nu_i\\

&= - 2 \sigma  \sum_i \nu_i \left(\omega_i -\Omega_i \right)\delta_{i,k} \\

&= - 2 \sigma  \nu_k \left(\omega_k -\Omega_k \right)
\end{aligned}
$$



$$
\begin{aligned}
-\frac{\partial }{\partial \omega_k } U(\omega, u, \lambda) 
 &= \partial_k \rho \sum_{j=0}^M \sum_i \left[ (u_j)_i \omega_i - \lambda_j \right]^2 \\
 &= \sum_{j=0}^M \sum_i^N2 \left((u_j)_i \omega_i - \lambda_j \right) (u_j)_i \delta_{ik} \\
 &= 2 \sum_{j=0}^M \left((u_j)_k \omega_k - \lambda_j \right) (u_j)_k
\end{aligned}
$$

### Exact solution
$$
\frac{\partial}{\partial \omega_k} (V + W + U) = 0
$$
In vector notation
$$
 0 = \vec{\mu} - \gamma \underline{\underline{\Theta}} \vec{\omega} -2 \sigma \cdot \text{diag}(\vec{\nu})\left( \vec{\omega} - \vec{\Omega} \right) + \rho \sum_j \underbrace{\vec{u}_j \odot \vec{\omega}  \odot \vec{u}_j}_{\text{diag}(\vec{u_j})^2 \vec{\omega}} - \lambda_j \vec{u}_j 
$$

$$
\vec{\omega} = \left( 2\rho \sum_j \text{diag}(\vec{u_j})^2 - \gamma \underline{\underline{\Theta}} -2 \sigma \cdot \text{diag}(\vec{\nu}) \right)^{-1} \left(2 \rho \sum_j \lambda_j \vec{u}_j + 2\sigma \text{diag}(\vec{\nu}) \vec{\Omega} - \vec{\mu}  \right)
$$

## Binary encoding

$$
\left\{1, 2, \dots, M \right\} \ni \omega_i = \sum_{q=0}^{[\log_2 M]} 2^{q-2}(s_q+1)
$$
or inversly
$$
s_j = 1-2\text{Heavyside} \left[ \omega \cdot 2^{1-j} -1 \right]
$$

$$
H(\omega) = \sum_{ij} J_{ij} \omega_i \omega_j = \sum_{ij} J_{ij} \left( \sum_{p}2^{p-2} (s_p+1) \right) \left(\sum_{q}2^{q-2} (s_q+1) \right)
$$

$$
\frac{\partial}{\partial s_k} H\left( \omega(s) \right) = \frac{\partial H}{\partial \omega_l}\frac{\partial \omega_l}{\partial s_k} = \left(\sum_i J_{il} \omega_i \right) \left( \partial_k \sum_{q=0}^{[\log_2 M]} 2^{q-2}(s_q+1) \right)
$$

$$
\partial_k H = \sum_i J_{il} \omega_i 2^{k-2} = \sum_i J_{il} 2^{k-2} \left(\sum_{q}2^{q-2} (s_q+1) \right)
$$

if $s_k$ is part of the representation of $\omega_l$, otherwise it is zero.

$\pm$

# Code

In [ ]:
using DifferentialEquations
using DynamicalSystems
using PlotlyJS, DataFrames
using IJulia
using SparseArrays
using Random
using LinearAlgebra
using StatsBase
using Polynomials, SpecialPolynomials

### Misc. functions

In [ ]:
function get_Jacobian(state_sizes)
    Jakobian = zeros(Float64, (N, M))
    q = 0

    for i in 1:N
        for j in 1:M
            if j <= sum(state_sizes[1:i])
                if i > 1
                    if  j > sum(state_sizes[1:i-1])
                        q = j - 1 - sum(state_sizes[1:i-1])
                        Jakobian[i, j] = 2.0^(q-2)
                    end
                else
                    q = j-1
                    Jakobian[i, j] = 2.0^(q-2)
                end
            end
        end
    end
    return sparse(Jakobian)
end

function get_ω_clipped(s, state_sizes, values, N, M)
    ω = zeros(N)
    s_clipped = [elem < 0 ? -1 : 1 for elem in s]
    idx = 0
    for i in 1:N
        ω_idx = 1 + Int(round( sum([2.0^(q-2)*(s_clipped[q+idx] + 1.0) for q in 1:state_sizes[i]]) ))
        if ω_idx > length(values[i])[1] # This has to be decided (maybe put random values from the allowed set)
            ω = 0
        else
            ω[i] = values[i][ω_idx] # ennek nincs értelem a második index 1:M_i' ig fut
        end
        idx += state_sizes[i]
    end
    return ω
end

function ω(s, state_sizes, values, N, M)
    ω = zeros(N)
    idx = 0
    for i in 1:N
        ω_cont = sum([ 2.0^(q-2)* (1.0+s[q+idx]) for q in 1:state_sizes[i]])
        ω_idx = Int(floor(ω_cont))
        #DEBUG
        if ω_idx < 0
            println(ω_cont)
            println(values[i])
            exit(86)
        end
        #DEBUG
        weight = ω_cont - ω_idx
        if ω_idx > state_sizes[i]
            ω[i] = values[i][end]
        else
            ω[i] = (1-weight) * values[i][ω_idx+1]
            if ω_idx+2 <= state_sizes[i]
                ω[i] += weight*values[i][ω_idx+2]
            end
        end
        idx += state_sizes[i]
    end
    return ω
end

function f_ω(ω, values, ϵ)
    if ω <= (values[1] - ϵ/2)
        return (ω - (values[1] - ϵ/2))^2
    elseif ω >= (last(values) + ϵ/2)
        return (ω - (last(values) + ϵ/2))^2
    else
        for j in 1:(size(values)[1])
            if ω > (values[j] - ϵ/2) && ω < (values[j] + ϵ/2)
                return -sqrt(0.25ϵ^2 -(ω - values[j])^2)
            elseif ω > (values[j] + ϵ/2) && ω < (values[j+1] - ϵ/2)
                return sqrt(0.25*(values[j+1] - values[j] - ϵ)^2 -(ω - 0.5*(values[j]+values[j+1]))^2)
            end
        end
    end
    return 0
end

function g_ω_old(ω, points, ϵ)
    N = size(ω)[1]
    result = zero(ω)
    for i in 1:N
        values = points[i]
        if ω[i] <= (values[1] - ϵ/2)
            result[i] = (ω[i] - (values[1] - ϵ/2))^2
        elseif ω[i] >= (last(values) + ϵ/2)
            result[i] = (ω[i] - (last(values) + ϵ/2))^2
        else
            for j in 1:(size(values)[1])
                if ω[i] > (values[j] - ϵ/2) && ω[i] < (values[j] + ϵ/2)
                    result[i] = -sqrt(0.25ϵ^2 -(ω[i] - values[j])^2)
                    break
                elseif ω[i] > (values[j] + ϵ/2) && ω[i] < (values[j+1] - ϵ/2)
                    result[i] = sqrt(0.25*(values[j+1] - values[j] - ϵ)^2 -(ω[i] - 0.5*(values[j]+values[j+1]))^2)
                    break
                else
                    result[i] = 0
                end
            end
        end
    end
    return result
end

function special_sawtooth(x, values, amplitude)
    N = size(x)[1]
    result = zero(x)

    for i in 1:N
        points = values[i]
        num_teeth = length(points) - 1
        if x[i] < points[1]
            result[i] = x[i] * amplitude / points[1]  # Linearly increasing function
        elseif x[i] > points[end]
            last_tooth_width = points[end] - points[end - 1]
            relative_x = x[i] - points[end]
            result[i] = (relative_x / last_tooth_width - 1) * amplitude  # Linearly decreasing function
        else
            for j in 1:num_teeth
                if x[i] >= points[j] && x[i] <= points[j + 1]
                    tooth_width = points[j + 1] - points[j]
                    relative_x = x[i] - points[j]
                    tooth_position = relative_x / tooth_width
                    result[i] = (2 * tooth_position - 1) * amplitude
                end
            end
        end
    end
    return result
end

function g_ω(a, ω, set_of_zeros)
    function σωσ(a, ω, α)
        return (2/(1 + exp(-a*(ω - α))) - 1)
    end

    product = 1.0
    if length(set_of_zeros) %2 == 1
        for i in 1:length(set_of_zeros)-1
            product *= σωσ(-a, ω, set_of_zeros[i]) * σωσ(a, ω,  0.5*(set_of_zeros[i]+set_of_zeros[i+1]))
        end
        return product * σωσ(-a, ω, set_of_zeros[end])
    else
        for i in 1:length(set_of_zeros)-1
            product *= σωσ(-a, ω, set_of_zeros[i]) * σωσ(a, ω,  0.5*(set_of_zeros[i]+set_of_zeros[i+1]))
        end
        return - product * σωσ(-a, ω, set_of_zeros[end])
    end
end

function get_Lagrange_poly(opo_nodes)
    peeks = Vector{Float64}()
    dist = opo_nodes[2] -opo_nodes[1]
    
    push!(peeks, maximum(opo_nodes)+0.1*dist)
    push!(peeks, minimum(opo_nodes)-0.1*dist)
    if length(opo_nodes) % 2 == 1
        push!(peeks, (opo_nodes[1]+opo_nodes[2])/2)
        push!(peeks, (opo_nodes[end]+opo_nodes[end-1])/2)
    else
        push!(peeks, (opo_nodes[Integer(length(opo_nodes)/2)] + opo_nodes[Integer(length(opo_nodes)/2)+1])/2 )
    end
    xs = Vector{Float64}()
    ys = Vector{Float64}()
    
    for i in 1:length(opo_nodes)
        push!(xs, opo_nodes[i])
        push!(ys, 0.0)
    end
    
    for i in 1:length(peeks)
        push!(xs, peeks[i])
        push!(ys, 1)
    end
    
    perm=sortperm(xs)
    xs = xs[perm]
    ys = ys[perm];

    return Lagrange(xs, ys)
end
    

### DEBUG

In [ ]:
s_points = [i for i in -1:0.01:1]

m_mat = zeros( (length(s_points), length(s_points)) )

s = -ones(M)

for i in 1:length(s_points)
    for j in 1:length(s_points)
        s[1:2] = [s_points[i], s_points[j]]
        m_mat[i, j] = ω(s, state_sizes, values, N, M)[1]
    end
end


In [ ]:
plot(heatmap(z=m_mat, x = s_points, y= s_points), Layout(xaxis_title="s1", yaxis_title="s2"))

In [ ]:
ωs = [i for i in 0:0.001:50]
vals = [5, 15, 20, 30, 45]
traces = [scatter(x = ωs, y = [g_ω(7.5, ω, vals) for ω in ωs]), 
        scatter(x = vals, y = zeros(length(vals)), mode="markers", marker_color="red")
        ];
layout = Layout(title="e_rhs",
                    xaxis_title="ω",
                    yaxis_title="e_rhs")
plot(traces, layout)

In [ ]:
opo_nodes = [10.0, 20, 30,  40,  50, 60.0]
peeks = Vector{Float64}()
dist = opo_nodes[2] -opo_nodes[1]

push!(peeks, maximum(opo_nodes)+0.1*dist)
push!(peeks, minimum(opo_nodes)-0.1*dist)
if length(opo_nodes) % 2 == 1
    #idx = Integer(floor(length(opo_nodes)/2))
    #push!(peeks, (opo_nodes[idx]+opo_nodes[idx+1])/2)
    #push!(peeks, (opo_nodes[idx+1]+opo_nodes[idx+2])/2)

    push!(peeks, (opo_nodes[1]+opo_nodes[2])/2)
    push!(peeks, (opo_nodes[end]+opo_nodes[end-1])/2)
else
    push!(peeks, (opo_nodes[Integer(length(opo_nodes)/2)] + opo_nodes[Integer(length(opo_nodes)/2)+1])/2 )
end

#opo_nodes = [1.0, 2.5, 4.0]
#peeks = [0.5, 1.5, 3.5, 4.5]

xs = Vector{Float64}()
ys = Vector{Float64}()

for i in 1:length(opo_nodes)
    push!(xs, opo_nodes[i])
    push!(ys, 0.0)
end

for i in 1:length(peeks)
    push!(xs, peeks[i])
    push!(ys, 1)
end

perm=sortperm(xs)
xs = xs[perm]
ys = ys[perm];

In [ ]:
p = Lagrange(xs, ys)

ωs = [i for i in 0.0:0.001:70]

traces = [scatter(x = ωs, y = [p(ω)*p(ω) for ω in ωs]), 
scatter(x = opo_nodes, y = zeros(length(opo_nodes)), mode="markers", marker_color="red", name="zeros"),
scatter(x = peeks, y = ones(length(peeks)), mode="markers", marker_color="green", name="peeks"),
        ];
layout = Layout(title="Vopo with Lagrange polynomials",
                    xaxis_title="ω",
                    yaxis_title="Vopo = (L(ω; peeks, nodes))²")
plot(traces, layout)

In [ ]:
function chbsv_scaling(points)
        return 2*(points ./ maximum(points)) - ones(size(points))
end

#xs_chbs = chbsv_scaling(xs)

p = Newton(xs, ys)

ωs = [i for i in 0.4:0.001:4.6]

traces = [scatter(x = ωs, y = [p(ω) for ω in ωs]), 
scatter(x = chbsv_scaling(opo_nodes), y = zeros(length(opo_nodes)), mode="markers", marker_color="red", name="zeros"),
scatter(x = chbsv_scaling(peeks), y = ones(length(peeks)), mode="markers", marker_color="green", name="peeks"),
        ];
layout = Layout(title="Vopo with Newton polynomials",
                    xaxis_title="ω",
                    yaxis_title="Vopo")
plot(traces, layout)

### Generating random input data

In [ ]:
N = 12

A = [1 0; 0 0]
B = [0 0; 0 1]


# Random connectivity matrix
J = 2* kron(A, rand(Int(N/2))) *kron(rand(Int(N/2)), A)' + 2 * kron( B, rand(Int(N/2), Int(N/2)) ) - kron( B, ones(Int(N/2), Int(N/2)) ) + 2 * kron(A, rand(Int(N/2), Int(N/2)) ) - kron(A, ones(Int(N/2), Int(N/2)) )
J = 0.5*(J + J')

ω_p = rand(N)
values = Array{Any}(undef, 0)
for i in 1:N
    unit = floor(rand() * 5) + 1
    upper = floor(rand() + unit)
    m_list = 10*[Int(j*unit) for j in 0:upper]
    push!(values, m_list)
end

# Random asset classes
state_sizes = [Int(ceil(log(2, size(val)[1]) )) for val in values]
M = sum(state_sizes)
values

In [ ]:
Jak_sparse = get_Jacobian(state_sizes)

### Update rule 1:

Equations of motion
$$
\frac{d}{dt} \vec{s} = \vec{e}(t) \circ \left[ J^T \partial_{\omega} H(\omega) \right] + \partial V_{opo}(s)
$$
$$
\frac{d}{dt} \vec{e} = -\beta \vec{e}(t) \circ \left[ \vec{s}(t) \circ \vec{s}(t) - \alpha \vec{1}\right]
$$
where $ \partial H = - \Theta \vec{\omega} (+ \vec{B}) $, and 
$$ \omega_j = \sum_{q=0}^{\left[ log_2 M \right]} 2^{q-2}([s_q] + 1) $$


In [ ]:
function update_rule1!(du, u, params, t)
    Jac_sparse  = params[1] # Sparse Jacobian matrix ω -> s
    J_ij        = params[2] # Connectivity matrix
    state_sizes = params[3] # Vector (of size N) bits per asset
    N           = params[4]
    M           = params[5]
    p           = params[6] # OPO pump parameter
    β           = params[7] # Feedback gain parameter
    α           = params[8] # Target amplitude parameter
    ω_function  = params[9] # Function to get ω from the state s
    values      = params[10]# Array of arrays containing the actual values of the assets

    # State variables
    e = u[M+1:2M]
    s = u[1:M]

    # N dimensional vector containing the values for the original state NOT SMOOTH
    ω           = ω_function(s, state_sizes, values, N, M)
    # N dimensional vector containing the gradient of the quadratic (Ising) potential
    ∂ωH         = -J_ij * ω
    # M dimensional vector, containing the OPO dynamics
    ∂V_opo      = (p-1)*s - (s .* s .* s)

    # Writing in output vector
    du[1:M] = e .* (Jac_sparse' * ∂ωH) + ∂V_opo
    du[M+1:2M] = -β * e .* ( (s .* s) - α * ones(M))
end

### Update rule 2:

Equations of motion:
$$
\frac{d}{dt} \vec{\omega} = \vec{e}\circ \partial_{\omega} H + \partial V_{opo} (\omega)
$$
$$
\frac{d}{dt} \vec{e} = -\beta \vec{e}\circ g\left( \omega \right)
$$
where $ \partial H = - \Theta \vec{\omega} (+ \vec{B}) $


In [ ]:
function update_rule2!(du, u, params, t)
    Jac_sparse  = params[1] # Dummy
    Θ           = params[2] # Connectivity matrix
    state_sizes = params[3] # Vector (of size N) bits per asset
    N           = params[4] # Characteristic problem size (nbr of assets)
    M           = params[5] # Dummy
    ∂V_opo      = params[6] # Gradient of the multy-stable OPO potential
    β           = params[7] # Feedback gain parameter
    ϵ           = params[8] # Parameter for the error variables
    g_ω         = params[9] # Function to get rhs of the error variables
    values      = params[10]# Array of arrays containing the acceptable values for each asset

    # State variables
    e = u[N+1:2N]
    ω = u[1:N]

    # N dimensional vector containing the gradient of the quadratic (Ising) potential
    ∂ωH         = - Θ * ω
 
    # Writing in output vector
    #du[1:N] = e .* ∂ωH + ∂V_opo
    du[1:N] = e .* ∂ωH + [∂V_opo[i](ω[i]) for i in 1:N]
    du[N+1:2N] = -β * e .* [g_ω(ϵ, ω[i], values[i]) for i in 1:N]
    #du[N+1:2N] .= 0.0
end

### Update rule 3:

Equations of motion:
$$
\frac{d}{dt} s = \vec{e}\circ \partial_{s} H + \partial V_{opo} (s)
$$
$$
\frac{d}{dt} \vec{e} = -\beta \vec{e}(t) \circ \left[ \vec{s}(t) \circ \vec{s}(t) - \alpha \vec{1}\right]
$$
where 
$$
 \partial_{s_k} H = - 2 \sum_{ij} \Theta_{ij} \omega_i \partial_k \omega_j 
$$


In [ ]:
function update_rule1!(du, u, params, t)
    Jac_sparse  = params[1] # Sparse Jacobian matrix ω -> s
    J_ij        = params[2] # Connectivity matrix
    state_sizes = params[3] # Vector (of size N) bits per asset
    N           = params[4]
    M           = params[5]
    p           = params[6] # OPO pump parameter
    β           = params[7] # Feedback gain parameter
    α           = params[8] # Target amplitude parameter
    get_ω       = params[9] # Function to get ω from the state s
    values      = params[10]# Array of arrays containing the actual values of the assets

    # State variables
    e = u[M+1:2M]
    s = u[1:M]

    # N dimensional vector containing the values for the original state NOT SMOOTH
    ω           = get_ω(s, state_sizes, values, N, M)
    # N dimensional vector containing the gradient of the quadratic (Ising) potential
    ∂sH         = -J_ij * ω
    # M dimensional vector, containing the OPO dynamics
    ∂V_opo      = (p-1)*s - (s .* s .* s)

    # Writing in output vector
    du[1:M] = e .* (Jac_sparse' * ∂ωH) + ∂V_opo
    du[M+1:2M] = -β * e .* ( (s .* s) - α * ones(M))
end

## Simulation 1

In [ ]:
tspan = (0.0, 2.0)

# Load data here
params = (Jak_sparse, J, state_sizes, N, M, 1.9, 0.15, 1.0, ω, values);

# initial condition(s)
u0 = ones(2M)
u0[1:M] = 2*rand(M)-ones(M)

# define problem and run simulation
prob = ODEProblem(update_rule1!, u0, tspan, params)
#sol = solve(prob, Euler(), dt = 0.01);
sol = solve(prob, Tsit5());

## Plotting 1

In [ ]:
traces = [scatter(x = sol.t, y = [vec[n] for vec in sol.u], name = string(n)) for n in 1:M];
layout1 = Layout(title="Softspins",
                   xaxis_title="t",
                   yaxis_title="Value")
p1() = plot(traces, layout1);

auxs = [scatter(x = sol.t, y = [vec[m] for vec in sol.u], name = string(m)) for m in M+1:2M];
layout2 = Layout(title="Error variables",
                   xaxis_title="t",
                   yaxis_title="Value")
p2() = plot(auxs, layout2);

energies = [ get_ω(s_t[1:M], state_sizes, values, N, M)'*(J *get_ω(s_t[1:M], state_sizes, values, N, M)) for s_t in sol.u]
layout3 = Layout(title="Ising Energy",
                   xaxis_title="t",
                   yaxis_title="Value")   

ω_trj = [ get_ω(s_t[1:M], state_sizes, values, N, M)' for s_t in sol.u]
traces2 = [scatter(x = sol.t, y = [ω[i] for ω in ω_trj], name = string(i)) for i in 1:N];
layout4 = Layout(title="ω",
                    xaxis_title="t",
                    yaxis_title="Value")
p4() = plot(traces2, layout4);


p3() = plot(scatter(x=sol.t, y= energies), layout3);
p = [p1() p2(); p3() p4()]



## Simulation 2

In [ ]:
tspan = (0.0, 15.0)

# Load data here
∂V_opo = [ -derivative(get_Lagrange_poly(nodes)^2) for nodes in values]
params = (0.0, J, state_sizes, N, M, ∂V_opo, -0.5, 10.0, g_ω, values);


# initial condition(s)
u0 = ones(2N)
u0[1:N] = [sample(f[2:end]) for f in values] + 2*rand(N) - ones(N)

# define problem and run simulation
prob = ODEProblem(update_rule2!, u0, tspan, params)
#sol = solve(prob, Euler(), dt = 0.01);
sol = solve(prob, Tsit5());

In [ ]:
traces = [scatter(x = sol.t, y = [vec[n] for vec in sol.u], name = string(n)) for n in 1:N];
layout1 = Layout(title="ω(t)",
                   xaxis_title="t",
                   yaxis_title="Value")
p1() = plot(traces, layout1);

auxs = [scatter(x = sol.t, y = [vec[m] for vec in sol.u], name = string(m-N)) for m in N+1:2N];
layout2 = Layout(title="Error variables",
                   xaxis_title="t",
                   yaxis_title="Value")
p2() = plot(auxs, layout2);

energies = [ s_t[1:N]'* J *s_t[1:N] for s_t in sol.u]
layout3 = Layout(title="Ising Energy",
                   xaxis_title="t",
                   yaxis_title="Value")   



p3() = plot(scatter(x=sol.t, y= energies), layout3);
p = [p1() p2() p3()]



In [ ]:
values

In [ ]:
ωs = [i for i in -1:0.001:71]
vals = [10, 20, 30, 40, 50, 60]
traces = [scatter(x = ωs, y = [g_ω(7.0, ω, vals) for ω in ωs]), 
        scatter(x = vals, y = zeros(length(vals)), mode="markers", marker_color="red")
        ];
layout = Layout(title="g(ω)",
                    xaxis_title="ω",
                    yaxis_title="g(ω)")
plot(traces, layout)